# Análisis de resultados: Meses 01-02 de 2015 para servicios de taxi YELLOW y GREEN

Para este análisis vamos a tomar en cuenta los dos primeros mese del año 2015. Cómo hallazgos, se encontró que hay carreras de diciembre del 2014 que fueron reportados en el primer mes del 2015. 

Ahora bien, para poder responder estas preguntas, vamos a utilizar el esquema de estrella (GOLD) que ya tenemos creado en snowflake. Para luego utilizar código SQL con las tablas 

## Prerequisitos

In [1]:
# snow_queries.py
import os
import sys
from typing import Optional
import pandas as pd
import snowflake.connector
from dotenv import load_dotenv

# carga .env si existe
load_dotenv()

def get_conn():
    user = "VICTOR18"
    pwd = "Vixtorbas2721633#"
    account = "CEDNWJG-BP62933"
    warehouse = "COMPUTE_WH"
    role = "ACCOUNTADMIN"
    database = "NYC_TLC"
    schema = "GOLD"

    missing = [k for k,v in [("SNOWFLAKE_USER", user), ("SNOWFLAKE_PASSWORD", pwd), ("SNOWFLAKE_ACCOUNT", account)] if v is None]
    if missing:
        raise RuntimeError(f"Faltan variables de entorno: {missing}")

    conn = snowflake.connector.connect(
        user=user,
        password=pwd,
        account=account,
        warehouse=warehouse,
        role=role,
        database=database,
        schema=schema,
        autocommit=True
    )
    return conn

def _fetch_pandas(cur):
    # Always use the manual DataFrame creation
    rows = cur.fetchall()
    cols = [d[0] for d in cur.description]
    return pd.DataFrame(rows, columns=cols)

def run_query(conn, sql: str) -> pd.DataFrame:
    cur = conn.cursor()
    try:
        cur.execute(sql)
        df = _fetch_pandas(cur)
        return df
    finally:
        cur.close()



In [2]:
# Test connection
def test_connection():
    conn = get_conn()
    try:
        # Simple query to test connection
        df = run_query(conn, "SELECT CURRENT_VERSION() as version")
        print("Connection successful!")
        print(f"Snowflake version: {df['VERSION'].iloc[0]}")
        return True
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return False
    finally:
        if conn:
            conn.close()

# Call this before your main code
test_connection()

Connection successful!
Snowflake version: 9.30.0


True

## Pregunta 1

**Demanda por zona y mes:** ¿cuáles son las 10 zonas con más viajes por mes? (PU y DO por separado).


In [3]:
def top10_zones_pickup(conn, month: Optional[int]=None, year: Optional[int]=None) -> pd.DataFrame:
    where = ""
    if year is not None:
        where += f" WHERE d.year = {int(year)}"
    if month is not None:
        where += (" AND" if where else " WHERE") + f" d.month = {int(month)}"

    sql = f"""
    WITH monthly AS (
      SELECT
        d.month,
        z.location_id,
        z.zone,
        COUNT(1) AS trips_count
      FROM NYC_TLC.GOLD.FCT_TRIPS f
      LEFT JOIN NYC_TLC.GOLD.DIM_ZONE z ON f.pu_zone_sk = z.zone_sk
      LEFT JOIN NYC_TLC.GOLD.DIM_DATE d ON f.pickup_date_sk = d.date_sk
      {where}
      GROUP BY d.month, z.location_id, z.zone
    )
    SELECT month, location_id, zone, trips_count
    FROM (
      SELECT monthly.*,
             ROW_NUMBER() OVER (PARTITION BY month ORDER BY trips_count DESC) AS rn
      FROM monthly
    )
    WHERE rn <= 10
    ORDER BY month, trips_count DESC;
    """
    return run_query(conn, sql)



In [4]:
def top10_zones_dropoff(conn, month: Optional[int]=None, year: Optional[int]=None) -> pd.DataFrame:
    where = ""
    if year is not None:
        where += f" WHERE d.year = {int(year)}"
    if month is not None:
        where += (" AND" if where else " WHERE") + f" d.month = {int(month)}"

    sql = f"""
    WITH monthly AS (
      SELECT
        d.month,
        z.location_id,
        z.zone,
        COUNT(1) AS trips_count
      FROM NYC_TLC.GOLD.FCT_TRIPS f
      LEFT JOIN NYC_TLC.GOLD.DIM_ZONE z ON f.do_zone_sk = z.zone_sk
      LEFT JOIN NYC_TLC.GOLD.DIM_DATE d ON f.dropoff_date_sk = d.date_sk
      {where}
      GROUP BY d.month, z.location_id, z.zone
    )
    SELECT month, location_id, zone, trips_count
    FROM (
      SELECT monthly.*,
             ROW_NUMBER() OVER (PARTITION BY month ORDER BY trips_count DESC) AS rn
      FROM monthly
    )
    WHERE rn <= 10
    ORDER BY month, trips_count DESC;
    """
    return run_query(conn, sql)

## Pregunta 2

**Ingresos y propinas:** ¿cómo varían los ingresos totales y el tip % por borough y
mes?

In [5]:
def revenue_and_tip_by_borough(conn, limit: int = 1000) -> pd.DataFrame:
    sql = f"""
    SELECT
      d.month,
      COALESCE(z.borough, 'UNKNOWN') AS borough,
      SUM(f.total_amount) AS total_revenue,
      SUM(f.tip_amount) AS total_tips,
      CASE WHEN SUM(f.total_amount) = 0 THEN 0 ELSE (SUM(f.tip_amount) / SUM(f.total_amount))*100 END AS tip_pct
    FROM NYC_TLC.GOLD.FCT_TRIPS f
    LEFT JOIN NYC_TLC.GOLD.DIM_ZONE z ON f.pu_zone_sk = z.zone_sk
    LEFT JOIN NYC_TLC.GOLD.DIM_DATE d ON f.pickup_date_sk = d.date_sk
    GROUP BY d.month, COALESCE(z.borough,'UNKNOWN')
    ORDER BY d.month, total_revenue DESC
    LIMIT {int(limit)};
    """
    return run_query(conn, sql)

## Pregunta 3

**Velocidad y congestión:** promedio de mph por franja horaria y borough (viajes diurnos
vs. nocturnos).


In [6]:
def avg_speed_by_period_and_borough(conn) -> pd.DataFrame:
    sql = """
    SELECT
      CASE WHEN t.hour BETWEEN 6 AND 17 THEN 'day' ELSE 'night' END AS day_period,
      COALESCE(z.borough,'UNKNOWN') AS borough,
      AVG(
        CASE 
          WHEN f.trip_duration_minutes > 0 
           AND f.trip_distance > 0
           AND f.trip_duration_minutes < 180  
           AND f.trip_distance < 50           
           AND (f.trip_distance / (f.trip_duration_minutes/60)) BETWEEN 1 AND 80  
          THEN (f.trip_distance) / (f.trip_duration_minutes/60)
          ELSE NULL
        END
      ) AS avg_mph,
      COUNT(1) AS total_trips,
      COUNT(
        CASE 
          WHEN f.trip_duration_minutes > 0 
           AND f.trip_distance > 0
           AND f.trip_duration_minutes < 180
           AND f.trip_distance < 50
           AND (f.trip_distance / (f.trip_duration_minutes/60)) BETWEEN 1 AND 80
          THEN 1 
        END
      ) AS valid_speed_trips
    FROM NYC_TLC.GOLD.FCT_TRIPS f
    LEFT JOIN NYC_TLC.GOLD.DIM_TIME t ON f.pickup_time_sk = t.time_sk
    LEFT JOIN NYC_TLC.GOLD.DIM_ZONE z ON f.pu_zone_sk = z.zone_sk
    GROUP BY
      CASE WHEN t.hour BETWEEN 6 AND 17 THEN 'day' ELSE 'night' END,
      COALESCE(z.borough,'UNKNOWN')
    ORDER BY borough, day_period;
    """
    return run_query(conn, sql)



## Pregunta 4

**Duración del viaje:** percentiles (p50/p90) de duración por PULocationID (pickup)


In [7]:
def trip_duration_percentiles_by_pu(conn, min_trips: int = 50, limit: int = 200) -> pd.DataFrame:
    sql = f"""
    SELECT
      z.location_id AS pu_location_id,
      z.zone,
      APPROX_PERCENTILE(f.trip_duration_minutes, 0.5) AS p50_minutes,
      APPROX_PERCENTILE(f.trip_duration_minutes, 0.9) AS p90_minutes,
      COUNT(1) AS trips_count
    FROM NYC_TLC.GOLD.FCT_TRIPS f
    LEFT JOIN NYC_TLC.GOLD.DIM_ZONE z ON f.pu_zone_sk = z.zone_sk
    GROUP BY z.location_id, z.zone
    HAVING COUNT(1) >= {int(min_trips)}
    ORDER BY trips_count DESC
    LIMIT {int(limit)};
    """
    return run_query(conn, sql)

## Pregunta 5

**Elasticidad temporal:** distribución de viajes por día de semana y hora; ¿cuáles son las horas pico?


In [8]:
def distribution_by_weekday_hour(conn, limit: int = 1000) -> pd.DataFrame:
    sql = f"""
    SELECT
        d.day_of_week + 1 AS day_of_week,
        CASE d.day_of_week
          WHEN 0 THEN 'Domingo'
          WHEN 1 THEN 'Lunes'
          WHEN 2 THEN 'Martes'
          WHEN 3 THEN 'Miercoles'
          WHEN 4 THEN 'Jueves'
          WHEN 5 THEN 'Viernes'
          WHEN 6 THEN 'Sabado'
        END AS day_desc,
        t.hour,
      COUNT(1) AS trips_count
    FROM NYC_TLC.GOLD.FCT_TRIPS f
    LEFT JOIN NYC_TLC.GOLD.DIM_DATE d ON f.pickup_date_sk = d.date_sk
    LEFT JOIN NYC_TLC.GOLD.DIM_TIME t ON f.pickup_time_sk = t.time_sk
    GROUP BY d.day_of_week, t.hour
    ORDER BY d.day_of_week, trips_count DESC
    LIMIT {int(limit)};
    """
    return run_query(conn, sql)

def top3_peak_hours_by_weekday(conn) -> pd.DataFrame:
    sql = """
    WITH agg AS (
      SELECT
        d.day_of_week + 1 AS day_of_week,
        CASE d.day_of_week
          WHEN 0 THEN 'Domingo'
          WHEN 1 THEN 'Lunes'
          WHEN 2 THEN 'Martes'
          WHEN 3 THEN 'Miercoles'
          WHEN 4 THEN 'Jueves'
          WHEN 5 THEN 'Viernes'
          WHEN 6 THEN 'Sabado'
        END AS day_desc,
        t.hour,
        COUNT(1) AS trips_count
      FROM NYC_TLC.GOLD.FCT_TRIPS f
      LEFT JOIN NYC_TLC.GOLD.DIM_DATE d ON f.pickup_date_sk = d.date_sk
      LEFT JOIN NYC_TLC.GOLD.DIM_TIME t ON f.pickup_time_sk = t.time_sk
      GROUP BY d.day_of_week + 1, day_desc, t.hour
    )
    SELECT day_of_week, day_desc, hour, trips_count
    FROM (
      SELECT agg.*,
             ROW_NUMBER() OVER (PARTITION BY day_of_week ORDER BY trips_count DESC) rn
      FROM agg
    )
    WHERE rn <= 3
    ORDER BY day_of_week, rn;
    """
    return run_query(conn, sql)



## Main de todas las respuestas

Primero, hacemos la conexión con snowflake y mandamos las tablas que necesitamos a csv's.

In [9]:
if __name__ == "__main__":
    conn = None
    try:
        conn = get_conn()

        # crear carpeta results si no existe
        results_dir = "results"
        os.makedirs(results_dir, exist_ok=True)

        # 1) Top 10 pickup por mes (sin filtrar)
        df1 = top10_zones_pickup(conn)
        print("TOP10 PICKUP (primeras filas):")
        print(df1.head())
        df1.to_csv(os.path.join(results_dir, "top10_pickup.csv"), index=False)

        # 1b) Top 10 dropoff por mes
        df1b = top10_zones_dropoff(conn)
        df1b.to_csv(os.path.join(results_dir, "top10_dropoff.csv"), index=False)

        # 2) Ingresos y tip% por borough y mes
        df2 = revenue_and_tip_by_borough(conn)
        df2.to_csv(os.path.join(results_dir, "revenue_by_borough.csv"), index=False)

        # 3) Velocidad promedio day/night por borough
        df3 = avg_speed_by_period_and_borough(conn)
        df3.to_csv(os.path.join(results_dir, "avg_speed_by_borough_period.csv"), index=False)

        # 4) P50 / P90 por pu_location_id
        df4 = trip_duration_percentiles_by_pu(conn)
        df4.to_csv(os.path.join(results_dir, "duration_percentiles_by_pu.csv"), index=False)

        # 5) Distribución por día/hora (y top 3 horas pico)
        df5 = distribution_by_weekday_hour(conn)
        df5.to_csv(os.path.join(results_dir, "distribution_by_weekday_hour.csv"), index=False)

        df5b = top3_peak_hours_by_weekday(conn)
        df5b.to_csv(os.path.join(results_dir, "top3_peak_hours_by_weekday.csv"), index=False)

        print("Consultas ejecutadas y CSVs generados en la carpeta 'results'.")
    except Exception as e:
        print("Error:", e, file=sys.stderr)
        raise
    finally:
        if conn:
            conn.close()


TOP10 PICKUP (primeras filas):
   MONTH  LOCATION_ID                       ZONE  TRIPS_COUNT
0      1          237      Upper East Side South      2884018
1      1          236      Upper East Side North      2770126
2      1          161             Midtown Center      2720804
3      1          162               Midtown East      2364300
4      1          230  Times Sq/Theatre District      2352603
Consultas ejecutadas y CSVs generados en la carpeta 'results'.


## Presentación de resultados

Ahora, creamos un templates para presentar las tablas obtenidas.

In [10]:
# Template: mostrar un solo CSV y opción de guardar la tabla completa
import os
import pandas as pd
from IPython.display import display

def mostrar_csv_template(fpath, max_full_rows=100, max_display_rows=50, save_full_html=True):
    """
    Muestra un CSV:
      - Si filas <= max_full_rows => muestra todo (display)
      - Si filas > max_full_rows => muestra head(max_display_rows) y, si save_full_html True, guarda HTML completo
    Retorna el DataFrame cargado.
    """
    fpath = os.path.abspath(fpath)
    if not os.path.exists(fpath):
        raise FileNotFoundError(f"No existe: {fpath}")
    df = pd.read_csv(fpath)
    nrows, ncols = df.shape
    print(f"Archivo: {os.path.basename(fpath)}")
    if nrows <= max_full_rows:
        display(df)
    else:
        print(f"Tabla grande: mostrando las primeras {max_display_rows} filas (de {nrows}).")
        display(df.head(max_display_rows))
        if save_full_html:
            out_html = os.path.splitext(fpath)[0] + "_full.html"
            try:
                df.to_html(out_html, index=False)
                print(f"Tabla completa guardada en: {out_html}")
            except Exception as e:
                print("Error guardando HTML completo:", e)
    return df

## Respuesta 1

In [11]:
df_1a = mostrar_csv_template("results/top10_pickup.csv", max_full_rows=200)


Archivo: top10_pickup.csv


,MONTH,LOCATION_ID,ZONE,TRIPS_COUNT
0,1,237,Upper East Side South,2884018
1,1,236,Upper East Side North,2770126
2,1,161,Midtown Center,2720804
3,1,162,Midtown East,2364300
4,1,230,Times Sq/Theatre District,2352603
...,...,...,...,...
115,12,142,Lincoln Square East,1995476
116,12,230,Times Sq/Theatre District,1982392
117,12,48,Clinton East,1927371
118,12,170,Murray Hill,1876178


In [12]:
df_1b = mostrar_csv_template("results/top10_dropoff.csv", max_full_rows=200)


Archivo: top10_dropoff.csv


,MONTH,LOCATION_ID,ZONE,TRIPS_COUNT
0,1,236,Upper East Side North,2916233
1,1,237,Upper East Side South,2589334
2,1,161,Midtown Center,2584065
3,1,170,Murray Hill,2173516
4,1,230,Times Sq/Theatre District,2086930
...,...,...,...,...
115,12,162,Midtown East,1860735
116,12,142,Lincoln Square East,1734101
117,12,48,Clinton East,1701123
118,12,239,Upper West Side South,1654826


## Respuesta 2

In [13]:
df_2 = mostrar_csv_template("results/revenue_by_borough.csv", max_full_rows=200)


Archivo: revenue_by_borough.csv


,MONTH,BOROUGH,TOTAL_REVENUE,TOTAL_TIPS,TIP_PCT
0,1,Manhattan,9.817170e+08,1.162406e+08,11.840544
1,1,Queens,2.332436e+08,2.536174e+07,10.873496
2,1,Brooklyn,5.366348e+07,4.698755e+06,8.755965
3,1,Unknown,1.863796e+07,2.114390e+06,11.344534
4,1,Bronx,8.919637e+06,2.225262e+05,2.494790
...,...,...,...,...,...
91,12,Unknown,1.714446e+07,1.926337e+06,11.235921
92,12,Bronx,8.396540e+06,2.440754e+05,2.906856
93,12,UNKNOWN,3.582594e+06,3.121738e+05,8.713625
94,12,EWR,6.602289e+05,7.669638e+04,11.616635


## Respuesta 3

In [14]:
df_3 = mostrar_csv_template("results/avg_speed_by_borough_period.csv", max_full_rows=200)


Archivo: avg_speed_by_borough_period.csv


,DAY_PERIOD,BOROUGH,AVG_MPH,TOTAL_TRIPS,VALID_SPEED_TRIPS
0,day,Bronx,13.238963,3183481,2948718
1,night,Bronx,15.932686,2076296,1906804
2,day,Brooklyn,11.544950,19472761,18840506
3,night,Brooklyn,14.311659,15458048,15040433
4,day,EWR,21.857510,52879,5752
5,night,EWR,22.067746,19934,2318
6,day,Manhattan,9.931828,469955369,464645693
7,night,Manhattan,12.701117,238600038,236045002
8,day,Queens,18.632112,46652346,44897360
9,night,Queens,21.295511,24220021,23370318


## Respuesta 4

In [15]:
df_4 = mostrar_csv_template("results/duration_percentiles_by_pu.csv", max_full_rows=200)


Archivo: duration_percentiles_by_pu.csv


,PU_LOCATION_ID,ZONE,P50_MINUTES,P90_MINUTES,TRIPS_COUNT
0,237,Upper East Side South,9.0,22.000000,31354025
1,161,Midtown Center,12.0,26.000000,29296718
2,236,Upper East Side North,9.0,23.000000,28580328
3,186,Penn Station/Madison Sq West,13.0,27.017143,25787297
4,162,Midtown East,11.0,25.000000,25711905
...,...,...,...,...,...
195,102,Glendale,17.0,42.000000,41882
196,3,Allerton/Pelham Gardens,18.0,50.000000,41519
197,117,Hammels/Arverne,44.0,79.000000,40383
198,200,Riverdale/North Riverdale/Fieldston,22.0,50.000000,39266


## Pregunta 5

## Pregunta 5a

In [16]:
df_5a = mostrar_csv_template("results/distribution_by_weekday_hour.csv", max_full_rows=200)


Archivo: distribution_by_weekday_hour.csv


,DAY_OF_WEEK,DAY_DESC,HOUR,TRIPS_COUNT
0,1,Domingo,10,6179143
1,1,Domingo,13,6140293
2,1,Domingo,9,6112338
3,1,Domingo,8,6047489
4,1,Domingo,11,6024000
...,...,...,...,...
163,7,Sabado,23,2910327
164,7,Sabado,3,2380530
165,7,Sabado,0,1605258
166,7,Sabado,2,1580346


## Pregunta 5b

In [17]:
df_5b = mostrar_csv_template("results/top3_peak_hours_by_weekday.csv")


Archivo: top3_peak_hours_by_weekday.csv


,DAY_OF_WEEK,DAY_DESC,HOUR,TRIPS_COUNT
0,1,Domingo,10,6179143
1,1,Domingo,13,6140293
2,1,Domingo,9,6112338
3,2,Lunes,14,7129301
4,2,Lunes,13,6950592
5,2,Lunes,15,6492967
6,3,Martes,14,7797570
7,3,Martes,13,7372600
8,3,Martes,15,7324204
9,4,Miercoles,14,8024958
